load dataset

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Load CSV
df = pd.read_csv("recsys_data.csv")

# if using reverst input/output
df.rename(columns={"input_seq": "target_seq", "target_seq": "input_seq"}, inplace=True)

# Keep only the desired columns
df = df[['input_seq', 'target_seq']]

df = df.dropna(subset=['input_seq', 'target_seq'])

# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to HuggingFace Dataset
train_data = Dataset.from_pandas(df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Combine into DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

train_data, valid_data = (
    dataset_dict["train"],
    dataset_dict["test"],
)

In [ ]:
def tokenize_example(example, max_length, sos_token, eos_token):
    input_tokens = [token for token in example["input_seq"].split()][:max_length]
    target_tokens = [token for token in example["target_seq"].split()][:max_length]

    input_tokens = [sos_token] + input_tokens + [eos_token]
    target_tokens = [sos_token] + target_tokens + [eos_token]
    return {"input_tokens": input_tokens, "target_tokens": target_tokens}

In [ ]:
max_length = 1024
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "max_length": max_length,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import tqdm

min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

input_toks = [unk_token, pad_token, sos_token, eos_token]
target_toks = [unk_token, pad_token, sos_token, eos_token]
for row in df.itertuples(index=False):
  target_seq = row.target_seq
  input_seq = row.input_seq
  for i in target_seq.split():
    if i not in target_toks:
      target_toks.append(i)
  for i in input_seq.split():
    if i not in input_toks:
      input_toks.append(i)

input_toks.sort()
target_toks.sort()

input_vocab = {tok:idx for idx, tok in enumerate(input_toks)}
target_vocab = {tok:idx for idx, tok in enumerate(target_toks)}



In [ ]:
def numericalize_example(example, input_vocab, target_vocab):
    input_ids = [input_vocab[i] for i in example["input_tokens"]]
    target_ids = [target_vocab[i] for i in example["target_tokens"]]
    return {"input_ids": input_ids, "target_ids": target_ids}


In [ ]:
fn_kwargs = {"input_vocab": input_vocab, "target_vocab": target_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)


In [ ]:
data_type = "torch"
format_columns = ["input_ids", "target_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)


In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_input_ids = [example["input_ids"] for example in batch]
        batch_target_ids = [example["target_ids"] for example in batch]
        batch_input_ids = nn.utils.rnn.pad_sequence(batch_input_ids, padding_value=pad_index)
        batch_target_ids = nn.utils.rnn.pad_sequence(batch_target_ids, padding_value=pad_index)
        batch = {
            "input_ids": batch_input_ids,
            "target_ids": batch_target_ids,
        }
        return batch

    return collate_fn



In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
batch_size = 256

unk_index = input_vocab['<unk>']
pad_index = input_vocab['<pad>']

train_data_loader = get_data_loader(train_data, batch_size, input_vocab['<pad>'], shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, input_vocab['<pad>'])


define model architecture

In [ ]:
'''
https://github.com/bentrevett/pytorch-seq2seq
'''
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

class Decoder(nn.Module):
  def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
      super().__init__()
      self.output_dim = output_dim
      self.hidden_dim = hidden_dim
      self.n_layers = n_layers
      self.embedding = nn.Embedding(output_dim, embedding_dim)
      self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
      self.fc_out = nn.Linear(hidden_dim, output_dim)
      self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, cell):
      input = input.unsqueeze(0)
      embedded = self.dropout(self.embedding(input))
      output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
      prediction = self.fc_out(output.squeeze(0))
      return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0, :]
        for t in range(1, trg_length):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        return outputs

In [ ]:
input_dim = len(input_vocab)
output_dim = len(target_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 768
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

training

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    loop = tqdm.tqdm(data_loader, desc="Training", leave=False)
    for i, batch in enumerate(loop):
        src = batch["input_ids"].to(device)
        trg = batch["target_ids"].to(device)
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)


def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    loop = tqdm.tqdm(data_loader, desc="Evaluating", leave=False)
    with torch.no_grad():
        for i, batch in enumerate(loop):
            src = batch["input_ids"].to(device)
            trg = batch["target_ids"].to(device)
            output = model(src, trg, 0)  # turn off teacher forcing
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

for epoch in range(last_epoch, n_epochs):
    print(f"Epoch: {epoch+1}")
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )

    #save last model
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch+1,
            'loss': train_loss,
        }, "model/recsys_model_last_reversed.pt")


    print(f"Train Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")

#validate after two epoch
valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )

print(f"Valid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")